In [142]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MONSTER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [143]:
df = pd.read_csv("spotify_millsongdata.csv")

In [144]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [145]:
df = df.sample(5000).drop('link',axis=1).reset_index(drop=True)

In [146]:
df.head(5)

,artist,song,text
0,Noa,All Is Well,"Another day, another small town \r\nA summer ..."
1,Backstreet Boys,Soldier,Wanna be there just in time \r\nWanna be your...
2,Styx,Young Man,Written by James Young \r\nLead vocals by Jam...
3,Ween,Happy Colored Marbles,"Most people are not ok, but they're taking the..."
4,Extreme,Slide,Find your groove \r\nThen drop the hammer \r...


In [147]:
df.shape

(5000, 3)

In [148]:
df.head(10)

,artist,song,text
0,Noa,All Is Well,"Another day, another small town \r\nA summer ..."
1,Backstreet Boys,Soldier,Wanna be there just in time \r\nWanna be your...
2,Styx,Young Man,Written by James Young \r\nLead vocals by Jam...
3,Ween,Happy Colored Marbles,"Most people are not ok, but they're taking the..."
4,Extreme,Slide,Find your groove \r\nThen drop the hammer \r...
5,Alabama,40 Hour Week,There are people in this country who work hard...
6,Dan Fogelberg,Heart Hotels,Well there's too many windows in this old hote...
7,Roxette,Keep Me Waiting,I hit the coast with the faith of a preacherma...
8,Tragically Hip,Summer's Killing Us,Does summer exist in its praises? \r\nDoes su...
9,Religious Music,I'll Walk With God,I'll Walk with God from this day on. \r\nHis ...


In [149]:
type(df)

pandas.DataFrame

In [150]:
df.shape

(5000, 3)

Text Cleaning

In [151]:
df['text'] = df['text'].str.lower().str.replace(r'^\W\S',' ', regex=True).str.replace(r'\n',' ', regex = True)

In [152]:
df

,artist,song,text
0,Noa,All Is Well,"another day, another small town \r a summer b..."
1,Backstreet Boys,Soldier,wanna be there just in time \r wanna be your ...
2,Styx,Young Man,written by james young \r lead vocals by jame...
3,Ween,Happy Colored Marbles,"most people are not ok, but they're taking the..."
4,Extreme,Slide,find your groove \r then drop the hammer \r ...
...,...,...,...
4995,Weird Al Yankovic,I Love Rocky Road,i hear those ice cream bells and i start to dr...
4996,George Harrison,Ooh Baby,"my baby, ooh \r you know i love you \r my ba..."
4997,Ocean Colour Scene,So Sad - The Riverboat Song B - Side,"so sad, so sad \r so sad, so sad \r i didn't..."
4998,Hank Williams Jr.,Ain't Nobody's Business,one day i have ham and bacon \r and the next ...


In [153]:
# Safely get index for "I'm Making Believe" (avoid IndexError if not present)
matches = df[df['song'] == "I'm Making Believe"].index
if len(matches) == 0:
    print("Song 'I'm Making Believe' not found in sampled dataframe. Showing candidates...")
    print(df[df['song'].str.contains("I'm Making Believe", case=False, na=False)]['song'].unique()[:20])
else:
    idx = matches[0]
    print('Index for I\'m Making Believe:', idx)

Index for I'm Making Believe: 4908


recommender function

In [154]:
stemmer = PorterStemmer()

In [155]:
def token(txt):
    if not isinstance(txt, str):
        return ""
    tokens = word_tokenize(txt)
    stems = [stemmer.stem(w) for w in tokens]
    return " ".join(stems)

In [156]:
token("you are beautiful,beauty")

'you are beauti , beauti'

In [157]:
df['text'].apply(lambda x: token(x))

0       anoth day , anoth small town a summer breez th...
1       wan na be there just in time wan na be your sh...
2       written by jame young lead vocal by jame young...
3       most peopl are not ok , but they 're take thei...
4       find your groov then drop the hammer overtak b...
                              ...                        
4995    i hear those ice cream bell and i start to dro...
4996    my babi , ooh you know i love you my babi , oo...
4997    so sad , so sad so sad , so sad i did n't want...
4998    one day i have ham and bacon and the next day ...
4999    oh thunderhead , you 're a man with a problem ...
Name: text, Length: 5000, dtype: str

In [158]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [159]:
tfid = TfidfVectorizer(analyzer='word', stop_words='english')

In [160]:
matrix=tfid.fit_transform(df['text'])

In [161]:
similer = cosine_similarity(matrix)

In [162]:
similer[0]

array([1.00000000e+00, 9.52916215e-04, 4.01415026e-03, ...,
       1.61498172e-03, 1.48232849e-02, 1.42832605e-03], shape=(5000,))

In [163]:
# Safely get index for 'Bottoms Up' (avoid IndexError if not present)
matches = df[df['song'] == "Bottoms Up"].index
if len(matches) == 0:
    print("Song 'Bottoms Up' not found in sampled dataframe. Showing candidates...")
    print(df[df['song'].str.contains("Bottoms Up", case=False, na=False)]['song'].unique()[:20])
else:
    idx = matches[0]
    print('Index for Bottoms Up:', idx)

Song 'Bottoms Up' not found in sampled dataframe. Showing candidates...
<StringArray>
[]
Length: 0, dtype: str


Recommender Function

In [164]:
def recommender(song_name, top_n=20):
    """Return top_n similar song titles for given song_name."""
    matches = df[df['song'] == song_name].index
    if len(matches) == 0:
        raise ValueError(f"Song '{song_name}' not found in dataframe")
    idx = matches[0]
    distances = list(enumerate(similer[idx]))
    distances = sorted(distances, reverse=True, key=lambda x: x[1])
    result = [df.iloc[i].song for i, _ in distances[1:top_n+1]]
    return result

In [165]:
# Test recommender with an existing song from the dataframe
test_song = df.iloc[0].song
test_song
recommender(test_song)

['One Phone Call',
 "If The Phone Doesn't Ring, It's Me",
 'Picture',
 'Here Comes II - New Version',
 'Daily Nightly',
 'What Would You Do',
 'Captain Nemo',
 "Just Rmember You're My Sunshine",
 'Banana Phone',
 'Love Story (Where Do I Begin)',
 'Bottle Rockets',
 'Another Rainy Night',
 'The King Is Dead',
 'Curious Thing',
 'Summer Breeze',
 'You Stopped Loving Me',
 "Movin' On Without You",
 'Stolen Car',
 'Crying',
 "For Pete's Sake"]

In [173]:
import pickle
import numpy as np

# Save CSV for easier compatibility
df.to_csv('music_data.csv', index=False)

# Save similarity matrix
np.save('similarity.npy', similer)

print('Saved music_data.csv and similarity.npy')

Saved music_data.csv and similarity.npy


In [167]:
import pickle

In [168]:
pickle.dump(similer, open("simiarity","wb")) 

In [169]:
pickle.dump(df, open("df", "wb"))